In [13]:
import os, glob, sqlite3
import pandas
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [14]:
os.makedirs("AI Research Paper Dataset", exist_ok=True)
Dataset_Path = os.path.join("AI Research Paper Dataset", "AI Research Paper Dataset.csv")

In [15]:
Dataset = pandas.read_csv(Dataset_Path)
Dataset

,Authors,Categories,Comment,DOI,Entry ID,Journal Ref,PDF URL,Primary Category,Published,Summary,Title,Updated
0,[arxiv.Result.Author('M. L. Ginsberg')],['cs.AI'],See http://www.jair.org/ for an online appendi...,NaN,http://arxiv.org/abs/cs/9308101v1,"Journal of Artificial Intelligence Research, V...",http://arxiv.org/pdf/cs/9308101v1,cs.AI,1993-08-01 00:00:00+00:00,Because of their occasional need to return to ...,Dynamic Backtracking,1993-08-01 00:00:00+00:00
1,[arxiv.Result.Author('M. P. Wellman')],['cs.AI'],See http://www.jair.org/ for any accompanying ...,NaN,http://arxiv.org/abs/cs/9308102v1,"Journal of Artificial Intelligence Research, V...",http://arxiv.org/pdf/cs/9308102v1,cs.AI,1993-08-01 00:00:00+00:00,Market price systems constitute a well-underst...,A Market-Oriented Programming Environment and ...,1993-08-01 00:00:00+00:00
2,"[arxiv.Result.Author('I. P. Gent'), arxiv.Resu...",['cs.AI'],See http://www.jair.org/ for any accompanying ...,NaN,http://arxiv.org/abs/cs/9309101v1,"Journal of Artificial Intelligence Research, V...",http://arxiv.org/pdf/cs/9309101v1,cs.AI,1993-09-01 00:00:00+00:00,We describe an extensive study of search in GS...,An Empirical Analysis of Search in GSAT,1993-09-01 00:00:00+00:00
3,"[arxiv.Result.Author('F. Bergadano'), arxiv.Re...",['cs.AI'],See http://www.jair.org/ for any accompanying ...,NaN,http://arxiv.org/abs/cs/9311101v1,"Journal of Artificial Intelligence Research, V...",http://arxiv.org/pdf/cs/9311101v1,cs.AI,1993-11-01 00:00:00+00:00,As real logic programmers normally use cut (!)...,The Difficulties of Learning Logic Programs wi...,1993-11-01 00:00:00+00:00
4,"[arxiv.Result.Author('J. C. Schlimmer'), arxiv...",['cs.AI'],See http://www.jair.org/ for an online appendi...,NaN,http://arxiv.org/abs/cs/9311102v1,"Journal of Artificial Intelligence Research, V...",http://arxiv.org/pdf/cs/9311102v1,cs.AI,1993-11-01 00:00:00+00:00,To support the goal of allowing users to recor...,Software Agents: Completing Patterns and Const...,1993-11-01 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,"[arxiv.Result.Author('Maxime Lefrançois'), arx...","['cs.CL', 'cs.AI']",NaN,NaN,http://arxiv.org/abs/1204.5316v1,MTT - 5th International Conference on Meaning-...,http://arxiv.org/pdf/1204.5316v1,cs.CL,2012-04-24 09:13:59+00:00,We are interested in bridging the world of nat...,ILexicOn: toward an ECD-compliant interlingual...,2012-04-24 09:13:59+00:00
9996,"[arxiv.Result.Author('Yoshiharu Maeno'), arxiv...","['q-fin.RM', 'cs.AI']",presented at the 4th World Congress on Social ...,NaN,http://arxiv.org/abs/1204.5661v2,presented at the 4th World Congress on Social ...,http://arxiv.org/pdf/1204.5661v2,q-fin.RM,2012-04-25 14:07:22+00:00,The European sovereign debt crisis has impaire...,Transmission of distress in a bank credit network,2012-11-22 08:59:47+00:00
9997,"[arxiv.Result.Author('C. Widanapathirana'), ar...","['cs.NI', 'cs.AI']",2011 Fourth IEEE International Conference on U...,10.1109/UCC.2011.42,http://arxiv.org/abs/1204.5805v1,Fourth IEEE International Conference on Utilit...,http://arxiv.org/pdf/1204.5805v1,cs.NI,2012-04-26 01:49:57+00:00,We present an automated solution for rapid dia...,Intelligent Automated Diagnosis of Client Devi...,2012-04-26 01:49:57+00:00
9998,"[arxiv.Result.Author('Paolo Liberatore'), arxi...","['cs.LO', 'cs.AI']",NaN,10.1016/j.ijar.2015.05.009,http://arxiv.org/abs/1204.5859v3,NaN,http://arxiv.org/pdf/1204.5859v3,cs.LO,2012-04-26 08:18:38+00:00,While looking for abductive explanations of a ...,On the Complexity of Finding Second-Best Abduc...,2015-05-14 13:11:02+00:00


In [16]:
def Authors_Name_Processing(Text):
    Name_Text = Text
    Name = []
    for i in range(len(Name_Text.split("'"))):
        if '.Author(' in Name_Text.split("'")[i]:
            Name.append(Name_Text.split("'")[i+1])
    return ", ".join(Name)

def Categories_Processing(Text):
    Categories_Text = Text
    Categories = []
    for i in Categories_Text.split("'"):
        if 'cs.' in i:
            Categories.append(i.strip('cs.'))
    return ", ".join(Categories)

a = Dataset.copy()

a['Authors'] = Dataset['Authors'].apply(lambda x: Authors_Name_Processing(x))
a['Categories'] = Dataset['Categories'].apply(lambda x: Categories_Processing(x))
a

,Authors,Categories,Comment,DOI,Entry ID,Journal Ref,PDF URL,Primary Category,Published,Summary,Title,Updated
0,M. L. Ginsberg,AI,See http://www.jair.org/ for an online appendi...,NaN,http://arxiv.org/abs/cs/9308101v1,"Journal of Artificial Intelligence Research, V...",http://arxiv.org/pdf/cs/9308101v1,cs.AI,1993-08-01 00:00:00+00:00,Because of their occasional need to return to ...,Dynamic Backtracking,1993-08-01 00:00:00+00:00
1,M. P. Wellman,AI,See http://www.jair.org/ for any accompanying ...,NaN,http://arxiv.org/abs/cs/9308102v1,"Journal of Artificial Intelligence Research, V...",http://arxiv.org/pdf/cs/9308102v1,cs.AI,1993-08-01 00:00:00+00:00,Market price systems constitute a well-underst...,A Market-Oriented Programming Environment and ...,1993-08-01 00:00:00+00:00
2,"I. P. Gent, T. Walsh",AI,See http://www.jair.org/ for any accompanying ...,NaN,http://arxiv.org/abs/cs/9309101v1,"Journal of Artificial Intelligence Research, V...",http://arxiv.org/pdf/cs/9309101v1,cs.AI,1993-09-01 00:00:00+00:00,We describe an extensive study of search in GS...,An Empirical Analysis of Search in GSAT,1993-09-01 00:00:00+00:00
3,"F. Bergadano, D. Gunetti, U. Trinchero",AI,See http://www.jair.org/ for any accompanying ...,NaN,http://arxiv.org/abs/cs/9311101v1,"Journal of Artificial Intelligence Research, V...",http://arxiv.org/pdf/cs/9311101v1,cs.AI,1993-11-01 00:00:00+00:00,As real logic programmers normally use cut (!)...,The Difficulties of Learning Logic Programs wi...,1993-11-01 00:00:00+00:00
4,"J. C. Schlimmer, L. A. Hermens",AI,See http://www.jair.org/ for an online appendi...,NaN,http://arxiv.org/abs/cs/9311102v1,"Journal of Artificial Intelligence Research, V...",http://arxiv.org/pdf/cs/9311102v1,cs.AI,1993-11-01 00:00:00+00:00,To support the goal of allowing users to recor...,Software Agents: Completing Patterns and Const...,1993-11-01 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,"Maxime Lefrançois, Fabien Gandon","CL, AI",NaN,NaN,http://arxiv.org/abs/1204.5316v1,MTT - 5th International Conference on Meaning-...,http://arxiv.org/pdf/1204.5316v1,cs.CL,2012-04-24 09:13:59+00:00,We are interested in bridging the world of nat...,ILexicOn: toward an ECD-compliant interlingual...,2012-04-24 09:13:59+00:00
9996,"Yoshiharu Maeno, Satoshi Morinaga, Hirokazu Ma...",AI,presented at the 4th World Congress on Social ...,NaN,http://arxiv.org/abs/1204.5661v2,presented at the 4th World Congress on Social ...,http://arxiv.org/pdf/1204.5661v2,q-fin.RM,2012-04-25 14:07:22+00:00,The European sovereign debt crisis has impaire...,Transmission of distress in a bank credit network,2012-11-22 08:59:47+00:00
9997,"C. Widanapathirana, J. Li, Y. A. Sekercioglu, ...","NI, AI",2011 Fourth IEEE International Conference on U...,10.1109/UCC.2011.42,http://arxiv.org/abs/1204.5805v1,Fourth IEEE International Conference on Utilit...,http://arxiv.org/pdf/1204.5805v1,cs.NI,2012-04-26 01:49:57+00:00,We present an automated solution for rapid dia...,Intelligent Automated Diagnosis of Client Devi...,2012-04-26 01:49:57+00:00
9998,"Paolo Liberatore, Marco Schaerf","LO, AI",NaN,10.1016/j.ijar.2015.05.009,http://arxiv.org/abs/1204.5859v3,NaN,http://arxiv.org/pdf/1204.5859v3,cs.LO,2012-04-26 08:18:38+00:00,While looking for abductive explanations of a ...,On the Complexity of Finding Second-Best Abduc...,2015-05-14 13:11:02+00:00
